## Data Pre-processing

This notebook iterates through each .abc tune book, extracting each tune and save as a new .abc file

### Folder-wise:

In [1]:
import os
import re

In [ ]:
folder = './Dataset/tune_books/CPartington' # Change to whichever folder you want to iterate through

# Iterate through all tune books:
for root, _, files in os.walk(folder):
    for name in files:
      file_path = os.path.join(root, name)
      if file_path.endswith('.abc'):
        destination_folder = './Dataset/tune_abc_files' # where to save to
        extension = '.abc'
        date = name[:4]
        file = open(file_path) # parse the tune book into python
        lines = file.readlines()


        # Iterate through all lines in the tune book:
        title = None
        abctune = []
        date_custom = None
        for i, line in enumerate(lines):
            line = line.strip()

            # Get the tune source:
            if line.startswith('link:'):
                source = 'S:' + line[5:]
            
            # Find each unique tune:
            unique_tune = (line.startswith('X:'))\
            or (line.startswith('W:') and i + 1 < len(lines) and lines[i+1].startswith('T:'))\
            or (line == '%%endtext')

            if unique_tune:
                
                # Save previous tune to .abc:
                if abctune:
                    if title:
                        if date_custom:
                            file_name = os.path.join(destination_folder, f"{date_custom}_{title}{extension}")
                        else:
                            file_name = os.path.join(destination_folder, f"{date}_{title}{extension}")
                        with open(file_name, 'w+') as f:
                            for l in abctune:
                                if re.match(r'^\s*$', l):
                                    # line is empty (has only the following: \t\n\r and whitespace)
                                    pass
                                else:
                                    f.write(l + '\n')

                # Clear previous tune and initialise a new one:
                date_custom = None
                count = 0
                title = None
                abctune = []
                abctune = [line]

            # Add each line to the tune:
            else:
                abctune.append(line)

                # Get the tune title:   
                if line.startswith('T:'):
                    count +=1
                    line = line.split('/')[0]
                    line = line[2:].strip()
                    if count == 1:
                        title = line
                        abctune.append(source)
                
                # Check if there is a commented year:
                if line.startswith('C:'):
                    match = re.search(r'.*([1-2][0-9]{3})', line)
                    if match:
                        date_custom = match.group(1)

        print(f"Written {name} to file")
      